하나투어 관광상품 리뷰 크롤링 분석


In [6]:
#only selenuim
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd

In [13]:
# selenium 웹 드라이버 서비스 설정
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = 'https://www.hanatour.com/trp/pkg/CHPC0PKG0200M200?pkgCd=AVP2092404017C1&prePage=major-products&directSale=PL00113904'
driver.get(url)

#여행 후기 클릭
#좀 더 안전하게 기다린 후에 크롤링
review_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ulTag"]/li[6]/a')))
review_link.click()
time.sleep(2)

#각 리뷰 정보를 저장할 빈 리스트
reviews = []

for page_num in range(1,13) : #1~12까지 반복
    #페이지네이션할 요소 찾기
    pagenate_div = driver.find_element(By.XPATH, '//*[@id="sticky06"]/div/div[4]/div[3]/div')
    
    #현재 몇 페이지 크롤링 중인지 출력
    print("현재 페이지: ", page_num)
    
    #li 태그 가져오기
    lis = driver.find_elements(By.CSS_SELECTOR, 'ul.list_review_v2 > li')

    for li in lis:
        #하나의 리뷰 정보를 저장할 딕셔너리
        review_info = {}
        rating_info = li.find_element(By.CLASS_NAME, 'rating_info')
        
        #별점: strong 태그 있으면 추출, 없으면 "ArithmeticError"
        if rating_info.find_element(By.TAG_NAME, 'strong'):
            rating = rating_info.find_element(By.TAG_NAME, 'strong').text 
        else:
            ""
        review_info['rating'] = rating
    
        #span 태그에 text를 차례대로 review_info의 각 항목에 저장
        spans = rating_info.find_elements(By.TAG_NAME, 'span')
        review_info['user'] = spans[1].text
        review_info['category'] = spans[2].text
        review_info['age'] = spans[3].text
        review_info['date'] = spans[-1].text
        
        #review_cont con 클래스에서 리뷰 추출
        review = li.find_element(By.CLASS_NAME, 'review_cont')
        if review:
            review_text = review.text
        else:
            ""
        review_info['review'] = review.text
        
        #review_cate 클래스에서 리뷰 카테고리 추출
        review_cate = li.find_element(By.CLASS_NAME,'review_cate')
        if review_cate:
            review_cate_text = review_cate.text
        else:
            ""
        review_info['review_cate'] = review_cate.text
        
        reviews.append(review_info)
    
    #다음 페이지로 이동
    try:
        #페이지가 1~9, 11~19, 21~29
        if page_num % 10 != 0:
            next_link = pagenate_div.find_element(By.XPATH, f"//span/a[text()='{page_num+1}']")
            next_link.click()
        #페이지가 10, 20, 30 ...
        else:
            next_link = pagenate_div.find_element(By.XPATH, '//*[@id="sticky06"]/div/div[4]/div[3]/div/div/a[2]')
            next_link.click()
        time.sleep(2)
    except:
        print('마지막 페이지에 도달했습니다.')
        break
        

#종료 
driver.quit()

#데이터프레임 생성
df = pd.DataFrame(reviews)
df

df.to_csv('tour_review.csv', index=False)

현재 페이지:  1
현재 페이지:  2
현재 페이지:  3
현재 페이지:  4
현재 페이지:  5
현재 페이지:  6
현재 페이지:  7
현재 페이지:  8
현재 페이지:  9
현재 페이지:  10
마지막 페이지에 도달했습니다.
